In [1]:
import pandas as pd
import plotly.express as px
import dash
import numpy as np
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import math
import time
from datetime import datetime

import requests

from ast import literal_eval
import dash_bootstrap_components as dbc

from sklearn.preprocessing import normalize # normalisation
from sklearn.preprocessing import MinMaxScaler

from pyaltmetric import Altmetric # altmetric


In [2]:
#read data
ss_df_reduced = pd.read_csv("./exports/ss_df_reduced.csv").drop(['influentialCitationCount'], axis=1)
weights = pd.read_csv("./exports/weights.csv")

min_idx=0
max_idx=1

In [3]:
minmax_df = pd.DataFrame({"min":ss_df_reduced.min(),"max":ss_df_reduced.max()}).T
minmax_df.drop("paperId", axis=1, inplace=True)
#minmax_df["referenceCount"][min_idx]    

In [4]:
#read data nutzlos
#dframe = pd.read_csv("citation_df.csv")
#df = dframe[:100]

In [5]:
app = dash.Dash(__name__)
colors = {
    'background': '#041014',
    'text': '#6674e4'
}


In [6]:
#Citations
def extractCategory(row):
    cat_list = []
    if row != None:
        for value in row:
            cat_list.append(value.get('category'))
    
    return cat_list

def getCitationDataFrame(paperId):
    offset = 0
    df = pd.DataFrame()
    start_ts = datetime.now().replace(microsecond=0)

    while offset != None:
        try:
            # print('Offset: '+str(offset))
            req = requests.get("https://api.semanticscholar.org/graph/v1/paper/"+paperId+"/citations?fields=title,referenceCount,citationCount,influentialCitationCount,isInfluential,fieldsOfStudy,s2FieldsOfStudy,year,publicationDate&limit=1000&offset="+str(offset)).json()
        
            # Wait 5 Minutes and retry if API-Server sends 'Too Many Requests'
            while req.get('message') == 'Too Many Requests':
                print('Waiting 5 minutes.')
                time.sleep((60*5)+5) # Wait 5 Minutes + 5 Extra-seconds for tolerance
                req = requests.get("https://api.semanticscholar.org/graph/v1/paper/"+paperId+"/citations?fields=title,referenceCount,citationCount,influentialCitationCount,isInfluential,fieldsOfStudy,s2FieldsOfStudy,year,publicationDate&limit=1000&offset="+str(offset)).json()
            
            df = pd.concat([df, pd.DataFrame(req.get('data'))])
            offset = req.get('next')
        except:
            print('An error was returned from the API-Server while querying the paper with paperId: ' + str(paperId) + '.. Proceeding with next row.')
        
    df = df[['isInfluential']].reset_index(drop=True).join(pd.json_normalize(df['citingPaper']))
    df['s2FieldsOfStudy'] = df['s2FieldsOfStudy'].apply(lambda x: extractCategory(x))

    # df['fieldsOfStudy'] = [ [] if x is None else x for x in df['fieldsOfStudy'] ] + df['s2FieldsOfStudy']
    df['fieldsOfStudy'] = [ [] if x is None or x is np.NaN else x for x in df['fieldsOfStudy'] ]

    # Combine foS & s2FoS
    df['fieldsOfStudy'] = df['fieldsOfStudy']+df['s2FieldsOfStudy']

    # Remove duplicates from the created list
    df['fieldsOfStudy'] = df['fieldsOfStudy'].apply(lambda row: sorted(list(dict.fromkeys(row))))

    # Remove rows with missing Ids
    if df[df['citationCount'].isna()].shape[0]>0:
        print('Removed ' + str(df[df['citationCount'].isna()].shape[0]) + ' rows which had missing paperId and external Ids.')
        df = df[~df['citationCount'].isna()]

    # Dtype conversion
    # df['year'] = df['year'].astype(int)
    df['referenceCount'] = df['referenceCount'].astype(int)
    df['citationCount'] = df['citationCount'].astype(int)
    df['influentialCitationCount'] = df['influentialCitationCount'].astype(int)

    end_ts = datetime.now().replace(microsecond=0)
    
    print('>>> Received ' + str(df.shape[0]) + ' rows [Total time elapsed: ' + str((end_ts-start_ts)) + ']')
    df.insert(0, 'Ref_paperId', paperId)
    
    return df.drop(['s2FieldsOfStudy'], axis=1)
    # return df
    # .insert(loc=0, column='Ref_paperId', value=paperId)

In [7]:
def getAMdata(paper_doi):

    try:
        # Altmetric-Objekt erstellen
        a = Altmetric()

        # In Altmetric-DB nach Paper mit entsprechender DOI suchen (API antwortet mit JSON-Array)
        altmetric_json_data = a.doi(paper_doi)

        # DataFrame erstellen und in geeignetes Format transformieren
        df = pd.DataFrame.from_dict(altmetric_json_data.items()).transpose()

        # DataFrame filtern
        df.columns = df.iloc[0]

        # Altmetric Score aufrunden
        df = df.iloc[1]
        df['score'] = np.ceil(df['score']).astype(int)

        # Alle Spalten extrahieren, die 'count' oder/und 'score' enthalten (z.B. gibt es nicht bei jedem Paper die Information wie viele Reddit, Wikipedia etc. Einträge vorhanden sind)
        col_list = []
        for col in df.keys():
            if any(x in col for x in ['count', 'score', 'doi']):
                col_list.append(col)

        # DataFrame nach den extrahierten Spalten filtern
        df = df[col_list]

        # Spalten umbennenen, damit zum Schluß nach dem Konkatenieren leichter nachvollzogen werden kann, aus welcher Quelle die Daten stammen.
        for col in df.keys():
            if col in col_list:
                df.rename({col:'altmetric_'+col}, inplace=True)
        return df
    except Exception as e:
        print(e)
        # return pd.DataFrame()
        pass

In [8]:
# Extrahiere Journalseiten, welche durch das Paper eingenommen werden
def extractOccupiedJournalPagesCount(journalPages):
    # Belegte Seiten des Journals in dem das Paper veröffentlicht wurde berechnen
    # Manche Angaben werden in ihrer Schreibweise verkürzt... z.B.: 153-57, dies soll wie 153-157 behandelt werden, da sonst negative Zahlen bei der folgenden Berechnung entstehen.

    try:
        split_pages = journalPages.strip().replace('–', '-').split('-')

        try:
            page_from = split_pages[0]
            page_to = split_pages[1]
            pagesOccupiedCount = int(page_to)-int(page_from)

            if pagesOccupiedCount<0:
                # Seitenzahl ist negativ (Seitenzahl von page_from war größer als Seitenzahl von page_to!)
                digit_delta = len(page_from)-len(page_to)
                # Z.B.: 133-38 wird geschaut aus wie vielen Zahlen beide aufgespaltenen Strings bestehen. Es wird die Länge verglichen, in dem Beispiel hat Zahl an [Index 0] 1 Stelle mehr als die Zahl an [Index 1].
                # Es wird nun von der Zahl an [Index 0] der Differenz entsprechend viele Zahlen von Links nach Rechts entnommen und an die Zahl an [Index 1] vorne angefügt.
                # So wird aus der Seitenzahl 38 nach diesem Muster [1]33 - [1]38 (die 1 der ersten Zahl wird vorne an die zweite Zahl angefügt).
                page_fixed = page_from[0:digit_delta]+page_to
                pagesOccupiedCount = int(page_fixed)-int(page_from)
                return pagesOccupiedCount
            else:
                return pagesOccupiedCount
                
        except:
            return 1

    except:
        return 0

In [9]:
def getSSdata(paperId):
  
  #######################
  #     SEND REQUEST    #
  #######################
  try:
    api_key = 'rGUKYOEpCP2FKQK88CLuB1izZBvDiQwA5SsSZ5vo'
    url = "https://api.semanticscholar.org/graph/v1/paper/"+paperId.replace(' ', '').strip()+"?fields=title,referenceCount,citationCount,influentialCitationCount,externalIds,authors.hIndex,authors.paperCount,authors.citationCount,authors.affiliations,authors.paperCount,authors.name,fieldsOfStudy,publicationTypes,publicationDate,year,journal,isOpenAccess"
    req = requests.get(url=url, params={'x-api-key': api_key}).json()

    # Wait 5 Minutes and retry if API-Server sends 'Too Many Requests'
    while req.get('message') == 'Too Many Requests':
      print('[Too Many Requests]: Waiting 5 minutes.')
      time.sleep((60*5)+5) # Wait 5 Minutes + 5 Extra-seconds for tolerance
      req = requests.get(url=url, params={'x-api-key': api_key}).json()

    ############################
    #     PROCESS RESPONSE     #
    ############################
    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in req.items() ]));

    # Features aus verschachteltem JSON-Array extrahieren und in separate Spalten teilen
    for col in ['DOI', 'CorpusId']:
      try:
        df[col] = df.loc[col].externalIds
      except:
        df[col] = np.nan
    
    df['paperId'] = df['paperId'].iloc[0]
    df['title'] = df['title'].iloc[0]
    df = df[df['externalIds'].isna()]
    df['journal_name'] = df.loc['name'].journal if df.loc['name'].journal else np.nan

    # Belegte Seiten des Journals in dem das Paper veröffentlicht wurde berechnen
    try:
        # Manche Angaben werden in ihrer Schreibweise verkürzt... z.B.: 153-57, dies soll wie 153-157 behandelt werden, da sonst negative Zahlen bei der folgenden Berechnung entstehen.
        df['occupiedJournalPages'] = extractOccupiedJournalPagesCount(df.loc['pages'].journal)
    except:
        df['occupiedJournalPages'] = 0

    # MV filtern
    df = df[~df['authors'].isna()][['DOI', 'CorpusId', 'paperId', 'title', 'referenceCount', 'citationCount', 'influentialCitationCount', 'fieldsOfStudy', 'publicationTypes', 'journal_name', 'occupiedJournalPages', 'isOpenAccess', 'publicationDate', 'year', 'authors']]

    # Author Name extrahieren
    df['author_names'] = df['authors'].apply(lambda x: x['name'])
    df['author_names'] = str(df['author_names'].to_list())
    df['author_names'] = df['author_names'].apply(literal_eval)

    # # Author Affiliation extrahieren
    # df['authors_affiliations'] = df['authors'].apply(lambda x: x['affiliations'])
    # df['authors_affiliations'] = str(df[~df['authors_affiliations'].isna()]['authors_affiliations'].to_list())
    # df['authors_affiliations'] = df['author_affiliations'].apply(literal_eval)

    # h-Index aller Autoren extrahieren
    df['hIndex'] = df['authors'].apply(lambda x: x['hIndex'])

    # Avg, min, max h-Index über alle Autoren berechnen, runden und in Ganzzahl umwandeln
    df['authors_sum_hIndex'] = df['hIndex'].sum().round(0).astype(int)
    # df['authors_mean_hIndex'] = df['hIndex'].mean().round(0).astype(int)
    # df['authors_max_hIndex'] = df['hIndex'].max().round(0).astype(int)
    # df['authors_min_hIndex'] = df['hIndex'].min().round(0).astype(int)

    # paperCount aller Autoren extrahieren
    df['author_paperCount'] = df['authors'].apply(lambda x: x['paperCount'])

    # Avg, min, max paperCount über alle Autoren berechnen, runden und in Ganzzahl umwandeln
    df['authors_sum_paperCount'] = df['author_paperCount'].sum().round(0).astype(int)
    # df['authors_mean_paperCount'] = df['author_paperCount'].mean().round(0).astype(int)
    # df['authors_max_paperCount'] = df['author_paperCount'].max().round(0).astype(int)
    # df['authors_min_paperCount'] = df['author_paperCount'].min().round(0).astype(int)

    # citationCount aller Autoren extrahieren
    df['author_citationCount'] = df['authors'].apply(lambda x: x['citationCount'])

    # Avg, min, max citationCount über alle Autoren berechnen, runden und in Ganzzahl umwandeln
    df['authors_sum_citationCount'] = df['author_citationCount'].sum().round(0).astype(int)
    # df['authors_mean_citationCount'] = df['author_citationCount'].mean().round(0).astype(int)
    # df['authors_max_citationCount'] = df['author_citationCount'].max().round(0).astype(int)
    # df['authors_min_citationCount'] = df['author_citationCount'].min().round(0).astype(int)

    df['authorsCount'] = len(df['author_names'])

    # Redundante bzw. irrelevante Features entfernen
    df = df.iloc[0][['DOI', 'CorpusId', 'paperId', 'title', 'author_names', 'isOpenAccess', 'publicationTypes', 'journal_name', 'occupiedJournalPages', 'publicationDate', 'year', 'fieldsOfStudy', 'referenceCount', 'citationCount', 'authorsCount', 'authors_sum_hIndex', 'authors_sum_paperCount', 'authors_sum_citationCount']]

    # Datentyp in Ganzzahl umwandeln
    df['referenceCount'] = df['referenceCount'].astype(int)
    df['citationCount'] = df['citationCount'].astype(int)
    # df['influentialCitationCount'] = df['influentialCitationCount'].astype(int)
    if df['year']:
      df['year'] = df['year'].astype(int)

    # print('- Collected Semantic Scholar metadata.')
    return df
  except Exception as e:
    print(e)
    return pd.DataFrame()

In [10]:
#Get paper information(Abfrage an Sementic, Altmetric, IEEE)
def getNewPaperWithDOI(input_string_doi):
    ss_df = pd.DataFrame(getSSdata(input_string_doi)).T
    try:
        al_df = pd.DataFrame(getAMdata(input_string_doi)).T
        ss_df = ss_df.merge(al_df, how='left', left_on='DOI', right_on='altmetric_doi').drop(['altmetric_doi'], axis=1)
    except:
        ss_df['altmetric_score'] = 0
    
    return ss_df


#normalization veraltet
def normaliseDataFrame(ss_df_reduced):
    df_normalized = normalize(ss_df_reduced[['referenceCount', 'citationCount', 'authorsCount','occupiedJournalPages', 'authors_sum_paperCount',
       'authors_sum_citationCount', 'authors_sum_hIndex', 'altmetric_score']])

    return ss_df_reduced[['paperId']].join(df_normalized)
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 100))
    return pd.DataFrame(scaler.fit_transform(data))


def calScore(npa):
    score = float(0.0)
    try:
        score += weights.referenceCount * ((npa["referenceCount"][0] - minmax_df["referenceCount"][min_idx])
                                           /(minmax_df["referenceCount"][max_idx]-minmax_df["referenceCount"][min_idx]))
        # print(score)
        score += weights.citationCount * ((npa["citationCount"][0] - minmax_df["citationCount"][min_idx])
                                           /(minmax_df["citationCount"][max_idx]-minmax_df["citationCount"][min_idx]))
        # print(score)
        score += weights.authorsCount * ((npa["authorsCount"][0] - minmax_df["authorsCount"][min_idx])
                                           /(minmax_df["authorsCount"][max_idx]-minmax_df["authorsCount"][min_idx]))
        # print(score)
        score += weights.occupiedJournalPages * ((npa["occupiedJournalPages"][0] - minmax_df["occupiedJournalPages"][min_idx])
                                           /(minmax_df["occupiedJournalPages"][max_idx]-minmax_df["occupiedJournalPages"][min_idx]))
        # print(score)
        score += weights.authors_sum_paperCount * ((npa["authors_sum_paperCount"][0] - minmax_df["authors_sum_paperCount"][min_idx])
                                           /(minmax_df["authors_sum_paperCount"][max_idx]-minmax_df["authors_sum_paperCount"][min_idx]))
        # print(score)
        score += weights.authors_sum_citationCount * ((npa["authors_sum_citationCount"][0] - minmax_df["authors_sum_citationCount"][min_idx])
                                           /(minmax_df["authors_sum_citationCount"][max_idx]-minmax_df["authors_sum_citationCount"][min_idx]))
        # print(score)
        score += weights.authors_sum_hIndex * ((npa["authors_sum_hIndex"][0] - minmax_df["authors_sum_hIndex"][min_idx])
                                           /(minmax_df["authors_sum_hIndex"][max_idx]-minmax_df["authors_sum_hIndex"][min_idx]))
        # print(score)
        try:
            score += weights.altmetric_score * ((npa["altmetric_score"][0] - minmax_df["altmetric_score"][min_idx])
                                            /(minmax_df["altmetric_score"][max_idx]-minmax_df["altmetric_score"][min_idx]))
        # print(score)
        except:
            pass
        # print(npa)
    except Exception as e:
        # print("################################")
        print(e)
        #multiply time 1000 cause of the minmax scaling
    return score*1000

#generate table
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

def generateFigure(doi):
    ss_df_nfig = pd.DataFrame(getSSdata(doi)).T
    
    paperCitations = getCitationDataFrame(doi)
    paperCitationsExploded = paperCitations.explode('fieldsOfStudy')
    
    newPFOS = paperCitationsExploded[paperCitationsExploded.year >= ss_df_nfig.year[0]].groupby('fieldsOfStudy')['fieldsOfStudy'].count().reset_index(name = 'CitationCount')
    newPYC = paperCitations[paperCitations.year >= ss_df_nfig.year[0]].groupby('year')['year'].count().reset_index(name = 'CitationCount')
    # print(newPYC)
    # print("new paper field of study")
    # print(newPFOS)
    figcitationcount = px.bar(newPYC, x='year', y='CitationCount', height=400)
    figfos = px.bar(newPFOS, x='fieldsOfStudy', y='CitationCount', height=400)
    print("exit generate fig")


In [11]:
npfirst = getNewPaperWithDOI('DOI:10.1001/jama.2020.2648')
npfirst["citationCount"]
#npfirst.columns

0    14441
Name: citationCount, dtype: object

In [12]:
#new_paper_=npfirst[minmax_df.columns]
#new_paper_["citationCount"][0]
#nsa = calScore(new_paper_)


In [13]:
paperCitations = getCitationDataFrame('10.1002/wps.20238')
paperCitationsExploded = paperCitations.explode('fieldsOfStudy')

#paperCitationsExploded

Removed 3 rows which had missing paperId and external Ids.
>>> Received 838 rows [Total time elapsed: 0:00:02]


In [14]:
ss_df_fig = pd.DataFrame(getSSdata('10.1002/wps.20238')).T
ss_df_fig.year

0    2015
Name: year, dtype: object

In [15]:
pcfos = paperCitationsExploded[paperCitationsExploded.year >= ss_df_fig.year[0]].groupby('fieldsOfStudy')['fieldsOfStudy'].count().reset_index(name = 'CitationCount')
# pcfos

In [16]:
pcyear = paperCitations[paperCitations.year >= ss_df_fig.year[0]].groupby('year')['year'].count().reset_index(name = 'CitationCount')
#pcyear

In [17]:
#pd.DataFrame(getSSdata('9d0c4389436d3ab8ed329dba8c58e8ac6737fd3b')).T
#ss_df_reduced.columns

In [18]:
#ss_df = pd.DataFrame(getSSdata('10.1002/wps.20238')).T
#al_df = pd.DataFrame(getAMdata('10.1002/wps.20238'))

In [19]:
npfirst = getNewPaperWithDOI('10.1002/wps.20238')
#npfirst

In [20]:
#citationCount per year
figcitationcount = px.bar(pcyear, x='year', y='CitationCount', height=400)
figfos = px.bar(pcfos, x='fieldsOfStudy', y='CitationCount', height=400)


In [21]:

# launch application

app.layout = html.Div([
    
  #  html.Div(children=[ html.Img(src='assets/oarilogo.png', style={'height': '60px','width':'60px'}), 
  #              html.H2(children='Online Attention Research Impact (OARI)', style={'textAlign': 'center', 'color': colors['text']})
   #                    
  #   ], style={'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center'}),
    #html.Img(src='assets/oarilogo.png', style={'height': '65px','position':'absolute','top':'10px','right':'100px'}),
    #html.Img(src='assets/frauaslogo.png', style={'height': '65px','position':'absolute','top':'10px','right':'300px'}),
 
    html.Div([
          html.Div([
             html.Img(src='assets/oarilogo.png', style={'height': '65px','width':'65px'})
          ],style={'alignItems':'center'}), 
         
          html.Div([
             html.H2(children='Online Attention Research Impact (OARI)', style={'textAlign': 'center', 'margin':'5px', 'color': colors['text']})
          ],style={'alignItems':'center'}), 
        html.Div([
             html.Img(src='assets/frauaslogo.png', style={'height': '65px'})
          ],style={'alignItems':'center'}),
                       
     ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}),
    
    html.Div([
    html.Div([
    
        html.Div([
            "Input: ",
            dcc.Input(id='input-string-doi', type='text', value='10.1002/wps.20238',debounce=True,style={'width':'250px','border': '3px solid #F9F7F9'}),
            html.Br(),
            
            html.Div(id='output-string'),
        ],style={'text-align':'center'}),
        
        html.Div([
         dcc.Loading(
            id="loading-1",
            type="default",
            children=[html.Div(id='output-1')]
        )
        ],style={'text-align':'center'}),

        html.Div([
         html.H2("Weight Information", style={'text-align': 'center', 'color': 'blue'}),


            html.P("Reference Count: {}".format(weights.iloc[0]['referenceCount'])),
            html.P("Citation Count: {}".format(weights.iloc[0]['citationCount'])),
            html.P("Authors Count: {}".format(weights.iloc[0]['authorsCount'])),
            html.P("Occupied Journal Pages: {}".format(weights.iloc[0]['occupiedJournalPages'])),
            html.P("Authors Sum of Paper Count: {}".format(weights.iloc[0]['authors_sum_paperCount'])),
            html.P("Authors Sum of Citation Count: {}".format(weights.iloc[0]['authors_sum_citationCount'])),
            html.P("Authors Sum of H-Index: {}".format(weights.iloc[0]['authors_sum_hIndex'])),
            html.P("Altmetric Score: {}".format(weights.iloc[0]['altmetric_score'])),
           ], style={'marginBottom': 50, 'marginTop': 25,'text-align': 'left',
      'border': '3px solid #F9F7F9'}),
        html.Br()
    ]), 
    
    html.Div([
         
        #html.H4(children='Figure 1', style={'textAlign': 'center'),
        dcc.Graph(id='citation-count-per-year', figure=figcitationcount),
        dcc.Graph(id='fieldofstudyperyear', figure=figfos)
    
    ]),
   ], style={'display': 'flex', 'border':'3px solid #F9F7F9'}),

    generate_table(ss_df_reduced)
], style={'background':'white','font-family': 'Segoe UI'})


In [22]:
@app.callback(Output(component_id='output-string', component_property='children'),
              [Input(component_id='input-string-doi', component_property='value')])
def update_output_string(input_string_doi):
    try:
        new_paper = getNewPaperWithDOI(input_string_doi) #getPaper
        #print(new_paper)
        npa = new_paper[minmax_df.columns]
        
        npa = npa.fillna(0)
       
        score = calScore(npa)
        
        #generateFigure(input_string_doi)
        
        print('Score was Calculated: ' + str(score[0]))
        return "Output: OARI-Index is "+str(math.ceil(score[0]))
    except Exception as e:
        print('Exception in "update_out_string!"')
        print(e)
        return 'The DOI could not be found, please try another one'

In [23]:
@app.callback(Output(component_id='citation-count-per-year', component_property='figure'),
              Output(component_id='fieldofstudyperyear', component_property='figure'),
              [Input(component_id='input-string-doi', component_property='value')])
def update_figure(input_string_doi):
   
    ss_df_nfig = pd.DataFrame(getSSdata(input_string_doi)).T
    
    paperCitations = getCitationDataFrame(input_string_doi)
    paperCitationsExploded = paperCitations.explode('fieldsOfStudy')
    
    newPFOS = paperCitationsExploded[paperCitationsExploded.year >= ss_df_nfig.year[0]].groupby('fieldsOfStudy')['fieldsOfStudy'].count().reset_index(name = 'CitationCount')
    newPFOS.sort_values("CitationCount", ascending = True, inplace=True)
    
    newPYC = paperCitations[paperCitations.year >= ss_df_nfig.year[0]].groupby('year')['year'].count().reset_index(name = 'CitationCount')
   # print(newPYC)
    figyearc = px.bar(newPYC, x='year', y='CitationCount', height=400, title="Citation count per Year")
    figfos = px.bar(newPFOS, x='fieldsOfStudy', y='CitationCount', color="CitationCount", height=400, title="Fields of Studies where Paper was cited")
    
    print("EXITING THE FIGURE UPDATE")
    return figyearc, figfos

In [24]:

@app.callback(Output(component_id='output-1', component_property='children'),
              [Input(component_id='input-string-doi', component_property='value')])
def update_output(input_string_doi):
    # print("enter in info call1")
    ss_info_df = pd.DataFrame(getSSdata(input_string_doi)).T
    # print(ss_info_df)
    # print("enter in info call2")
    return html.Div([
        html.H2("Paper Information", style={'text-align': 'center', 'color': 'blue'}),
        html.P('Paper Id: {}'.format(ss_info_df['paperId'][0])),
        html.P('Title: {}'.format(ss_info_df['title'][0])),
        html.P('Names of Authors: {}'.format(", ".join(ss_info_df['author_names'][0]))),
        html.P('Published in: {}'.format(ss_info_df['year'][0])),
        html.P('Fields Of Study: {}'.format(ss_info_df['fieldsOfStudy'][0])),
        html.P('Reference Count: {}'.format(ss_info_df['referenceCount'][0]))
    ], style={'marginBottom': 50, 'marginTop': 25,'text-align': 'left',
      'border': '3px solid #F9F7F9'})


In [25]:
if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [13/Feb/2023 17:31:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /assets/oarilogo.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /assets/frauaslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 17:31:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Feb/2023 17:31:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:15] "POST /_dash-update-component HTTP/1.1" 200 -


Score was Calculated: 8.892454820688839


127.0.0.1 - - [13/Feb/2023 17:31:17] "POST /_dash-update-component HTTP/1.1" 200 -


Removed 3 rows which had missing paperId and external Ids.
>>> Received 838 rows [Total time elapsed: 0:00:02]
EXITING THE FIGURE UPDATE


127.0.0.1 - - [13/Feb/2023 17:31:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2023 17:31:29] "POST /_dash-update-component HTTP/1.1" 200 -


Score was Calculated: 1342.4277377312058
Removed 5 rows which had missing paperId and external Ids.


127.0.0.1 - - [13/Feb/2023 17:32:26] "POST /_dash-update-component HTTP/1.1" 200 -


>>> Received 8995 rows [Total time elapsed: 0:00:58]
EXITING THE FIGURE UPDATE
